In [1]:
%%bash
pip install onnx onnxruntime onnxscript torch torch_geometric 
pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cpu.html

ModuleNotFoundError: No module named 'onnx'

In [13]:
from torch_geometric.utils import scatter
import torch_geometric

import torch
import torch.nn as nn
from torch_scatter import scatter_add
from typing import Optional, Dict
import torch_scatter

from torch.utils.checkpoint import checkpoint
import onnx
import onnxruntime as ort
import onnxscript
from onnxscript import opset18  # opset 18 is the latest (and only) supported version for now

print("Torch version:", torch.__version__)
print("torch-scatter version:", torch_scatter.__version__)
print("ONNX version:", onnx.__version__)
print("onnxruntime version:", ort.__version__)
print("torch_geometric version:", torch_geometric.__version__)

torch.random.manual_seed(42);

Torch version: 2.6.0
torch-scatter version: 2.1.2
ONNX version: 1.17.0
onnxruntime version: 1.20.1
torch_geometric version: 2.6.1


In [4]:
onnx_registry = torch.onnx.OnnxRegistry()
print(f"aten::gelu.default is supported by ONNX registry: \
    {onnx_registry.is_registered_op(namespace='aten', op_name='gelu', overload='default')}")

/Users/xju/miniconda3/envs/jupyterbook/lib/python3.12/site-packages/torch/onnx/_internal/_exporter_legacy.py:101: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/Users/xju/miniconda3/envs/jupyterbook/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/Users/xju/miniconda3/envs/jupyterbook/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


aten::gelu.default is supported by ONNX registry:     True


In [11]:
num_spacepoints = 100
spacepoint_features = 3
num_edges = 2000

x = torch.rand(num_spacepoints, spacepoint_features).to(torch.float32)
edge_list = torch.randint(0, 100, (2, num_edges)).to(torch.int64)
row, col = edge_list

num_edge_features = 4
edge_attr = torch.rand(num_edges, num_edge_features).to(torch.float32)

out = torch.cat([x[row], x[col], edge_attr], 1).float()
print("edge features shape:", out.shape)

edge features shape: torch.Size([2000, 10])


In [57]:
x2 = torch.rand(num_spacepoints, spacepoint_features).to(torch.float32)
edge_list2 = torch.randint(0, 100, (2, num_edges)).to(torch.int64)
row2, col2 = edge_list2
edge_attr2 = torch.rand(num_edges, num_edge_features).to(torch.float32)
out2 = torch.cat([x2[row2], x2[col2], edge_attr2], 1).float()

In [29]:
agg_mean = scatter(out, col, dim=0, dim_size=x.size(0), reduce='mean')
agg_sum = scatter(out, col, dim=0, dim_size=x.size(0), reduce='sum')
agg_max = scatter(out, col, dim=0, dim_size=x.size(0), reduce='max')
print("agg mean:", agg_mean.shape, agg_mean[0, :3])
print("agg sum:", agg_sum.shape, agg_sum[0, :3])
print("agg max:", agg_max.shape, agg_max[0, :3])

agg mean: torch.Size([100, 10]) tensor([0.5400, 0.4889, 0.4279])
agg sum: torch.Size([100, 10]) tensor([11.3401, 10.2661,  8.9863])
agg max: torch.Size([100, 10]) tensor([0.9344, 0.9712, 0.9005])


In [16]:
agg_sum2 = torch.scatter_add(torch.zeros(x.size(0), out.size(1)), 0, col.unsqueeze(1).expand(-1, out.size(1)), out)
print("agg sum2:", agg_sum2.shape, agg_sum2[0, :3])

agg sum2: torch.Size([100, 10]) tensor([11.3401, 10.2661,  8.9863])


In [20]:
agg_scatter = torch.scatter(torch.zeros(x.size(0), out.size(1)), 0, col.unsqueeze(1).expand(-1, out.size(1)), out, reduce="add")
print("scatter: ", agg_scatter.shape, agg_scatter[0, :3])

scatter:  torch.Size([100, 10]) tensor([11.3401, 10.2661,  8.9863])


/var/folders/v_/rpz3sncx2fg57t_m4n2_hkz40000gn/T/ipykernel_72441/838719481.py:1: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:234.)
  agg_scatter = torch.scatter(torch.zeros(x.size(0), out.size(1)), 0, col.unsqueeze(1).expand(-1, out.size(1)), out, reduce="add")


In [32]:
index = col.unsqueeze(1).expand(-1, out.size(1))
agg_mean3 = torch.scatter_reduce(torch.zeros(x.size(0), out.size(1)), 0, index=index, src=out, reduce="mean", include_self=False)
agg_sum3 = torch.scatter_reduce(torch.zeros(x.size(0), out.size(1)), 0, index=index, src=out, reduce="sum")
agg_max3 = torch.scatter_reduce(torch.zeros(x.size(0), out.size(1)), 0, index=index, src=out, reduce="amax")
print("agg mean3:", agg_mean3.shape, agg_mean3[0, :3])
print("agg sum3:", agg_sum3.shape, agg_sum3[0, :3])
print("agg max3:", agg_max3.shape, agg_max3[0, :3])

agg mean3: torch.Size([100, 10]) tensor([0.5400, 0.4889, 0.4279])
agg sum3: torch.Size([100, 10]) tensor([11.3401, 10.2661,  8.9863])
agg max3: torch.Size([100, 10]) tensor([0.9344, 0.9712, 0.9005])


In [33]:
tolerance = 1e-5
assert torch.allclose(agg_mean, agg_mean3, atol=tolerance)
assert torch.allclose(agg_sum, agg_sum3, atol=tolerance)
assert torch.allclose(agg_max, agg_max3, atol=tolerance)

In [47]:
class ModelPyGScatter(torch.nn.Module):
    def forward(self, index: torch.Tensor, src: torch.Tensor, size: int, reduce: str) -> torch.Tensor:
        return scatter(src, index, dim=0, dim_size=size, reduce=reduce)

In [48]:
model = ModelPyGScatter()
model.eval()
# model = torch.jit.script(model)
torch.onnx.export(model, (col, out, x.size(0), "mean"), "pyg_scatter.onnx", opset_version=18, dynamo=True)

W0131 11:14:23.251000 72441 site-packages/torch/onnx/_internal/exporter/_registration.py:66] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `ModelPyGScatter()` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ModelPyGScatter()` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'pkg.onnxscript.torch_lib.common': 1, '': 18, 'pkg.onnxscript.torch_lib': 1},
            producer_name='pytorch',
            producer_version='2.6.0',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"index"<INT64,[2000]>,
                %"src"<FLOAT,[2000,10]>
            ),
            outputs=(
                %"div"<FLOAT,[100,10]>
            ),
        ) {
             0 |  # node_Constant_0
                  %"val_0"<?,?> ⬅️ ::Constant() {value=Tensor<INT64,[1]>(array([100]), name=None)}
             1 |  # node_ConstantOfShape_1
                  %"val_1"<?,?> ⬅️ ::ConstantOfShape(%"val_0")
             2 |  # node_CastLike_2
                  %"new_zeros"<FLOAT,[100]> ⬅️ ::CastLike(%"val_1", %"src")
             3 |  # node_Constant_3
                  %"val_2"<?,?> ⬅️ ::Constant() {value_

In [64]:
def test_scatter(onnx_file: str, index, src):
    m = onnx.load(onnx_file)
    sess = ort.InferenceSession(onnx_file)
    print("input size: ", len(m.graph.input))
    # print(m.graph.input)
    inputs = {
        m.graph.input[0].name: index.numpy(),
        m.graph.input[1].name: src.numpy(),
        # m.graph.input[2].name: x.size(0),
        # m.graph.input[3].name: reduce
    }
    return torch.from_numpy(sess.run([m.graph.output[0].name], inputs)[0])

In [65]:
ort_result = test_scatter("pyg_scatter.onnx", col, out)
print("ort_result:", ort_result.shape, ort_result[0, :3])

input size:  2
ort_result: torch.Size([100, 10]) tensor([0.5400, 0.4889, 0.4279])


In [66]:
ort_result2 = test_scatter("pyg_scatter.onnx", col2, out2)
agg_mean_v2 = scatter(out2, col2, dim=0, dim_size=x2.size(0), reduce='mean')
print("ort_result2:", ort_result2.shape, ort_result2[0, :3])
print("agg_mean_v2:", agg_mean_v2.shape, agg_mean_v2[0, :3])
assert torch.allclose(agg_mean_v2, ort_result2, atol=tolerance)

input size:  2
ort_result2: torch.Size([100, 10]) tensor([0.5586, 0.4802, 0.4576])
agg_mean_v2: torch.Size([100, 10]) tensor([0.5586, 0.4802, 0.4576])


In [76]:
class ModuleHyperEdgeFinding(nn.Module):
    def forward(self, x, hyperedge_index, r: int = 2, message_feats: int = 3):
        hyperedge_index = hyperedge_index.permute(dims=(1,0))
        x_hyper = torch.gather(
            x.unsqueeze(1).expand([-1,r,-1]),
            0,
            hyperedge_index.unsqueeze(2).expand([-1, -1, message_feats])
        ).transpose(1,2)
        return x_hyper.sum(2)

In [67]:
def __hyperedge_finding__(x, hyperedge_index, r: int, message_feats: int):
    hyperedge_index = hyperedge_index.permute(dims=(1,0))
    x_hyper = torch.gather(
        x.unsqueeze(1).expand([-1,r,-1]),
        0,
        hyperedge_index.unsqueeze(2).expand([-1, -1, message_feats])
    ).transpose(1,2)
    return x_hyper.sum(2)

In [ ]:
def new_hperedge_finding(x, hyperedge_index, r: int, message_feats: int):
    # Unpack node indices and hyperedge indices
    node_idx, hyperedge_idx = hyperedge_index

    # Create an output tensor for aggregated hyperedge features
    hyperedge_feats = torch.zeros(hyperedge_idx.max() + 1, x.shape[1])

    # Scatter node features into hyperedge slots and sum them
    hyperedge_feats.scatter_reduce_(
        dim=0, 
        index=hyperedge_idx.unsqueeze(-1).expand(-1, x.shape[1]), 
        src=x[node_idx], 
        reduce="sum"
    )


In [69]:
edge_finding = __hyperedge_finding__(x, edge_list, 2, 3)
print(edge_finding.shape, edge_finding[0, :3])

torch.Size([2000, 3]) tensor([1.0591, 0.4294, 0.8407])


In [79]:
model = ModuleHyperEdgeFinding()
model.eval()
results = model(x, edge_list)
print(results.shape, results[0, :3])

# export to onnx
model = torch.jit.script(model)
torch.onnx.export(model, (x, edge_list, 2, 3), "hyperedge_finding.onnx", opset_version=18, dynamo=False)

torch.Size([2000, 3]) tensor([1.0591, 0.4294, 0.8407])


In [81]:
# export to onnx with dynamo
model = ModuleHyperEdgeFinding()
model.eval()
torch.onnx.export(model, (x, edge_list, 2, 3), "hyperedge_finding_dynamo.onnx", opset_version=18, dynamo=True)

W0131 11:49:53.206000 72441 site-packages/torch/onnx/_internal/exporter/_registration.py:66] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `ModuleHyperEdgeFinding()` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ModuleHyperEdgeFinding()` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'pkg.onnxscript.torch_lib.common': 1, '': 18, 'pkg.onnxscript.torch_lib': 1},
            producer_name='pytorch',
            producer_version='2.6.0',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"x"<FLOAT,[100,3]>,
                %"hyperedge_index"<INT64,[2,2000]>
            ),
            outputs=(
                %"sum_1"<FLOAT,[2000,3]>
            ),
        ) {
             0 |  # node_Transpose_0
                  %"permute"<INT64,[2000,2]> ⬅️ ::Transpose(%"hyperedge_index") {perm=[1, 0]}
             1 |  # node_aten_unsqueeze_1
                  %"unsqueeze"<FLOAT,[100,1,3]> ⬅️ pkg.onnxscript.torch_lib::aten_unsqueeze(%"x") {dim=1}
             2 |  # node_Constant_2
                  %"val_0"<?,?> ⬅️ ::Constant() {value=Tensor<INT64,[3]>(array([-1,  2, -1]), name=None)}
             